<a href="https://colab.research.google.com/github/Wittgenbot/fine-tuning/blob/main/finetuned_model_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up

## Package Installs

In [1]:
! pip install -q -U accelerate
! pip install -q -U bitsandbytes
! pip install -q -U huggingface-hub
! pip install -q -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

## Imports

In [2]:
import textwrap

import huggingface_hub
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# Model

## Configuration

In [3]:
model_name = 'descartesevildemon/Wittgenbot-7B'

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type='nf4',
                                bnb_4bit_use_double_quant=True)

# Download Model

In [4]:
save_directory = model_name.replace('/','_')

huggingface_hub.snapshot_download(repo_id=model_name,
                                  local_dir_use_symlinks=False,
                                  local_dir=save_directory)

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

'/content/descartesevildemon_Wittgenbot-7B'

## Load Model

In [5]:
model = AutoModelForCausalLM.from_pretrained(save_directory,
                                             quantization_config=bnb_config,
                                             torch_dtype=torch.bfloat16,
                                             device_map='auto',
                                             local_files_only=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Load and Configure Tokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained(save_directory)

tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = 'right'
tokenizer.add_eos_token = False
tokenizer.add_bos_token = False

# Model Manual Tests

## Response Generation Function

In [7]:
def query_model(question):

  prompt_template = """
  <s>
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about Ludwig Wittgenstein in a conversational, clear and coherent tone:
  {question}
  [/INST]
  """
  prompt = prompt_template.format(question=question)

  encodeds = tokenizer(prompt,
                       return_tensors="pt",
                       add_special_tokens=True)

  model_inputs = encodeds.to('cuda:0')

  generated_ids = model.generate(**model_inputs,
                                 max_new_tokens=1000,
                                 do_sample=True,
                                 top_p=0.9,
                                 temperature=0.7,
                                 pad_token_id=tokenizer.eos_token_id)

  decoded = tokenizer.batch_decode(generated_ids)

  response = decoded[0]

  formatted_response = ''
  lines = response.split('\n')
  for line in lines:
      if len(line) > 125:
          w = textwrap.TextWrapper(width=125, break_long_words=False)
          line = '\n'.join(w.wrap(line))
      formatted_response += line + '\n'

  print(formatted_response)

## Manual Tests

In [8]:
question ='According to Ludwig Wittgenstein\'s philosophy, is the existence of a private language possible?'
query_model(question)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  According to Ludwig Wittgenstein's philosophy, is the existence of a private language possible?
  [/INST]
  
  Wittgenstein's philosophy, particularly as expressed in his later work, such as the Philosophical Investigations, strongly
argues against the possibility of a private language. A private language, in this context, is a language that only one person
uses, and whose meanings are known only to that person. Wittgenstein claims that such a language is impossible for several
reasons.

Firstly, he argues that the meaning of a word is its use in the language. If a word is only used by one person, how can it
have a meaning? The person using the word might have a private mental state associated with it, but this does not constitute
a meaning in the sense that other people can understa

In [9]:
question ='What determines the meaning of a word?'
query_model(question)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  What determines the meaning of a word?
  [/INST]
  
  In the philosophy of Ludwig Wittgenstein, the meaning of a word is not determined by a single definition or a private
mental image, but rather by its use in the context of language games. Wittgenstein introduced the concept of language games
to emphasize that language is a tool used in various social contexts, and the meaning of a word arises from its usage within
these contexts.

For Wittgenstein, meaning is not an inherent property of a word, but rather something that emerges from the complex web of
social practices, agreements, and rules that govern language use. He famously stated, "the meaning of a word is its use in
the language" (Philosophical Investigations, §43), implying that understanding the meaning of a word requires 

In [32]:
question = 'How did Wittgenstein use the example of "games" to illustrate the family resemblance concept?'
query_model(question)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  How did Wittgenstein use the example of "games" to illustrate the family resemblance concept?
  [/INST]
  
  Wittgenstein's concept of "family resemblance" is a key idea in his philosophy, particularly in his later work, such as the
"Philosophical Investigations." He used the example of "games" to illustrate this concept.

Wittgenstein argued that many concepts, such as "games," do not have a fixed, definite essence that makes them all the same.
Instead, they have a series of similarities and overlaps, much like members of a family. Each member of the family shares
some characteristics with others, but no one characteristic is shared by all.

When Wittgenstein discussed games, he pointed out that there are various types of games, such as board games, card games,
ball games, and so on

In [11]:
question ='How do philosophical problems arise from misunderstandings of language?'
query_model(question)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  How do philosophical problems arise from misunderstandings of language?
  [/INST]
  
  Wittgenstein, in his later philosophy, argued that many philosophical problems arise from misunderstandings of language. He
believed that we often use language in a misleading way, assuming that words have fixed, definitive meanings, when in fact,
the meaning of a word is determined by its use in a particular language game.

For Wittgenstein, a language game is a specific context or activity in which language is used. For example, the language game
of giving and following orders is different from the language game of describing an object. Each game has its own rules, and
the meaning of a word is determined by these rules and the role it plays in the game.

When we fail to recognize the specific lan

In [12]:
question ='Can you provide examples of how philosophical problems arise from the misuse of language?'
query_model(question)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  Can you provide examples of how philosophical problems arise from the misuse of language?
  [/INST]
  
  Absolutely, I'd be happy to explain that! Ludwig Wittgenstein, in his later philosophy, particularly in the Philosophical
Investigations, argued that many philosophical problems arise from our misuse of language.

For instance, consider the problem of other minds. We can't directly perceive the minds of others, only their behavior. This
leads to the philosophical question of whether we can truly know that other people have minds and feelings similar to our
own. However, Wittgenstein would argue that this problem arises from a misunderstanding of the concept of 'mind'. We use the
word 'mind' in many different ways, such as 'having a mind of one's own', 'giving someone a piece of yo

In [13]:
question ='In what ways does the examination of pain\'s sameness challenge the understanding of subjective experiences?'
query_model(question)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  In what ways does the examination of pain's sameness challenge the understanding of subjective experiences?
  [/INST]
  
  That's a great question! Wittgenstein, in his later philosophy, particularly in the "Philosophical Investigations,"
challenges our understanding of subjective experiences, including pain, by examining the concept of sameness.

Wittgenstein argues that our understanding of concepts like pain is largely based on our ability to recognize similarities
and differences in our experiences. We learn to use the word "pain" by observing others' behavior when they are in pain, and
then applying that concept to our own experiences.

However, when it comes to the concept of pain's sameness, Wittgenstein points out that our experiences of pain are inherently
subjective and uni

In [14]:
question ='What is the purpose of the analogy with the toolbox?'
query_model(question)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  What is the purpose of the analogy with the toolbox?
  [/INST]
  
  The purpose of the analogy with the toolbox in Ludwig Wittgenstein's philosophy is to illustrate the concept of language
games and how words acquire meaning through use and context, as presented in his later philosophical work, particularly in
the Philosophical Investigations.

Wittgenstein uses the analogy of a toolbox to explain that words, like tools, have different uses depending on the context or
"game" in which they are used. Just as a hammer is used for driving nails, a screwdriver is used for turning screws, and a
wrench is used for tightening bolts, words are used differently in various language games.

For instance, the word "game" can refer to a competitive activity (in the language game of sports), a form

In [15]:
question ='What does following a rule entail?'
query_model(question)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  What does following a rule entail?
  [/INST]
  
  Following a rule, in the context of Ludwig Wittgenstein's philosophy, particularly as expressed in his later work, is not
about following a set of strict, predetermined steps or procedures. Instead, it's about understanding the concept or 'form of
life' that the rule is a part of, and then acting in accordance with that understanding.

Wittgenstein, in his "Philosophical Investigations," challenges the idea that there are fixed, universal rules that govern
our actions. He argues that the meaning of a rule is not inherent in the rule itself, but is rather determined by how it is
used in a particular context or 'form of life.'

So, when we talk about following a rule, we're not just talking about a set of instructions that we can follow

In [16]:
question ='Can you describe Wittgenstein\'s picture theory of meaning?'
query_model(question)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  Can you describe Wittgenstein's picture theory of meaning?
  [/INST]
  
  Absolutely, I'd be happy to explain Wittgenstein's picture theory of meaning. This theory is presented in his early work,
the Tractatus Logico-Philosophicus.

The picture theory of meaning suggests that language functions like a picture of the world. Just as a picture is a model of
reality, language is a model of reality in the form of propositions. These propositions, according to Wittgenstein, can
depict the world's states of affairs.

Wittgenstein argues that the structure of a proposition mirrors the structure of the fact it represents. This is similar to
how a picture's structure corresponds to the structure of the object it depicts. For instance, a picture of a tree has
branches, just as a tree has branch

In [17]:
question ='What does Wittgenstein say about tautologies and contradictions in the Tractatus?'
query_model(question)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  What does Wittgenstein say about tautologies and contradictions in the Tractatus?
  [/INST]
  
  Wittgenstein, in his Tractatus Logico-Philosophicus, discusses tautologies and contradictions as part of his exploration of
the relationship between language, logic, and reality. Tautologies and contradictions are significant in his work because
they highlight the limits of meaningful discourse and the importance of propositions being connected to the world.

A tautology, according to Wittgenstein, is a statement that is true in all possible circumstances. It is a sentence that does
not provide any new information about the world since its truth is a mere consequence of its logical structure. For example,
the statement "Either it is raining or it is not raining" is a tautology. Wittgenste

In [18]:
question ='Why is the notion of a criterion for correctness important for Wittgenstein\'s private language argument?'
query_model(question)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  Why is the notion of a criterion for correctness important for Wittgenstein's private language argument?
  [/INST]
  
  The notion of a criterion for correctness is important for Wittgenstein's private language argument because it addresses
the problem of solipsism and the possibility of a private language. A criterion for correctness refers to the standards or
rules that determine the accuracy or truth of a statement or belief.

In the private language argument, Wittgenstein challenges the idea that individuals can have a private language, meaning a
language that only they understand and use. He argues that for language to be meaningful, it must be public and shareable, as
the meaning of words and concepts is derived from their use within a linguistic community.

The notion of a cri

In [19]:
question ='What does Wittgenstein say about aspect perception?'
query_model(question)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  What does Wittgenstein say about aspect perception?
  [/INST]
  
  Wittgenstein, in his later philosophy, particularly in the "Philosophical Investigations," discusses the concept of aspect
perception, which refers to the ability to see something as something else, or to experience a shift in the way we perceive
or understand something.

He uses the example of seeing a drawing as a duck or a rabbit, depending on how we're prompted to see it. This illustrates
the idea that our perception is not solely determined by the object itself, but also by our mental set, or the concepts and
categories we bring to the perception.

Wittgenstein argues that this phenomenon is not a matter of having different images in our mind, but rather a difference in
the way we organize and interpret the same 

In [20]:
question ='What is the paradox of ostension?'
query_model(question)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  What is the paradox of ostension?
  [/INST]
  
  The paradox of ostension is a philosophical issue that arises in Ludwig Wittgenstein's later philosophy, particularly in
his work "Philosophical Investigations." It's a problem related to the way we use language to refer to things in the world.

Ostension is a way of referring to something by pointing to it. For example, if I point at a tree and say "tree," I'm using
ostension. But Wittgenstein noticed that this process isn't as straightforward as it seems.

The paradox arises when we consider what happens if someone who doesn't understand the word "tree" sees me point at a tree
and say "tree." They might learn to use the word in this context, but they still wouldn't understand what a tree is. They'd
just know that "tree" is the word w

In [21]:
question ='What is Wittgenstein trying to show with the builders\' language example?'
query_model(question)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  What is Wittgenstein trying to show with the builders' language example?
  [/INST]
  
  Wittgenstein, in his later philosophy, particularly in the "Philosophical Investigations," uses the builders' language
example to illustrate his idea of language games and the meaning of words. In this example, he describes a scenario where two
builders are working together, using a rudimentary form of language to coordinate their actions. One builder calls out
various commands, such as "slab," "pillar," or "block," to which the other builder responds by bringing the corresponding
object.

Wittgenstein is trying to show that the meaning of words is not inherent or determined by some external reality, but rather
by the use and context within a language game. In this case, the words "slab," "pillar,

In [22]:
question = 'What did Wittgenstein show using the "five red apples" example?'
query_model(question)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  What did Wittgenstein show using the "five red apples" example?
  [/INST]
  
  Wittgenstein's "five red apples" example is from his later philosophical period, particularly from the Philosophical
Investigations. The example is used to illustrate his concept of language games and meaning.

In this example, Wittgenstein asks us to imagine a situation where someone points to a group of five red apples and says
"five red apples." According to Wittgenstein, the meaning of the phrase "five red apples" is not inherent in the words
themselves, but rather it arises from the use of the words in the specific context of this language game.

The language game includes the rules of how words are used in this particular situation. For instance, the rule here might be
that "five red apples" refers t